In [1]:
import os
from pathlib import Path

# Print initial working directory
print("Initial working directory:", os.getcwd())

# Define the project root (one level up from current script location)
PROJECT_ROOT = Path("..").resolve()

# Set the working directory to the main project directory
os.chdir(PROJECT_ROOT)

# Verify if the directory has been set correctly
print("Updated working directory:", os.getcwd())

Initial working directory: c:\Users\Kiko\Desktop\Code\tutoring\Group-Work-Favorita-Forecasting\notebooks
Updated working directory: C:\Users\Kiko\Desktop\Code\tutoring\Group-Work-Favorita-Forecasting


In [2]:
from Favorita_TSA.viz.color_manager import ColorManager
from Favorita_TSA.viz.ploty_export import save_all

color_manager = ColorManager()

In [3]:
from typing import Any

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Uses your dynamic manager
from Favorita_TSA.viz.color_manager import ColorManager, apply_modern_theme


def plot_full_color_audit():
    """
    Fully automated audit: Detects groups, individual colors, and lists
    directly from COLORS.yaml without manual assignment.
    """
    # Reset and load the raw dictionary for iteration
    ColorManager._colors = None
    ColorManager._raw_dict = None
    raw_data = ColorManager.get_raw_dict()

    # Filter main groups (e.g., theme_dark, analysis, visualizations)
    if raw_data is not None:
        groups: dict[str, Any] = {
            k: v for k, v in raw_data.items() if isinstance(v, dict)
        }
    else:
        groups = {}

    # Dynamic calculation of row count based on YAML structure
    fig = make_subplots(
        rows=len(groups),
        cols=1,
        vertical_spacing=0.1,
        subplot_titles=[f"<b>Group: {k.upper().replace('_', ' ')}</b>" for k in groups],
    )

    for i, (group_name, content) in enumerate(groups.items(), 1):
        labels, values = [], []

        # Internal helper to flatten nested YAML structures
        def parse_content(data: Any, prefix: str = ""):
            if isinstance(data, dict):
                for k, v in data.items():  # type: ignore
                    parse_content(v, f"{prefix}{k} ")
            elif isinstance(data, list):
                # Automatically detect lists like Top 20, Top 10, etc.
                for idx, color in enumerate(data):  # type: ignore
                    labels.append(f"Index {idx+1}")  # type: ignore # noqa: B023
                    values.append(color)  # type: ignore # noqa: B023
            else:
                # Individual color (Hex code)
                labels.append(prefix.strip())  # noqa: B023
                values.append(data)  # noqa: B023

        parse_content(content)

        # Add bars
        fig.add_trace(
            go.Bar(
                x=labels,  # type: ignore
                y=[1] * len(labels),  # type: ignore
                marker_color=values,
                # Numerical hex codes inside bars (Standard 2025-10-29)
                text=values,  # type: ignore
                textposition="auto",
                # Auto-rotation if there are many colors
                textangle=-90 if len(labels) > 8 else 0,  # type: ignore
                textfont={"size": 9},
                name=group_name,
            ),
            row=i,
            col=1,
        )

    # Layout Optimization
    fig.update_layout(
        # Automatically scale height with the number of groups
        height=200 * len(groups),
        showlegend=False,
        title_text="Global Design System: Automated Audit",
        margin={"t": 80, "b": 50},
    )

    # Remove unnecessary axis labels for a clean look
    fig.update_yaxes(showticklabels=False, range=[0, 1.2])

    apply_modern_theme(fig)
    fig.show()
    # Note: Ensure save_all is defined or imported in your environment
    save_all(fig, "config/color_themes")


if __name__ == "__main__":
    plot_full_color_audit()